* 选用不同特征集
* 重新划分测试集进行测试
    * 训练集01 2013-01 至 2017-07, 测试集 2017-08
    * 训练集02 2013-01 至 2017-08, 测试集 2017-09
    * 训练集03 2013-01 至 2017-09, 测试集 2017-10
* 后面再重新划分训练集, 最好选用相同时长, 测试结果最佳为宜
* 测试 XGBoost/LightGBM

# 选用带时间窗口, 带同比/环比数据集 特征集测试

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import scipy as sp
from sklearn.model_selection import train_test_split, cross_val_score, KFold
import lightgbm as lgb
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [2]:
data = pd.read_csv('../../raw/LiChuan/dataset.csv')
submit = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

In [3]:
# 去掉 2012 年数据
data = data[data['year'] != 2012]
labels = data['sale_quantity']
data = data.drop(['sale_date', 'class_id', 'sale_quantity'], axis=1)

In [4]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = data[:-560]
test_1 = data[-560:-420]
labels_1 = labels[:-560]
test_labels_1 = labels[-560:-420]

# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = data[:-420]
test_2 = data[-420:-280]
labels_2 = labels[:-420]
test_labels_2 = labels[-420:-280]

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = data[:-280]
test_3 = data[-280:-140]
labels_3 = labels[:-280]
test_labels_3 = labels[-280:-140]

## 测试 LightGBM

In [5]:
lgb_train_1 = lgb.Dataset(train_1, labels_1)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_1 = lgb.train(params, lgb_train_1, 900)
lgb_pred_1 = model_1.predict(test_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - lgb_pred_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 120.66188708082196


In [10]:
lgb_train_2 = lgb.Dataset(train_2, labels_2)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_2 = lgb.train(params, lgb_train_2, 900)
lgb_pred_2 = model_2.predict(test_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - lgb_pred_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 116.8883026326931


In [7]:
lgb_train_3 = lgb.Dataset(train_3, labels_3)

params = {
'learning_rate': 0.003,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 30,
'min_hessian': 1,
'verbose': -1,}

model_3 = lgb.train(params, lgb_train_3, 900)
lgb_pred_3 = model_3.predict(test_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - lgb_pred_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 131.56512907867213


In [8]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 123.03843959739572


* 加上同比/环比 特征 LightGBM 测试的效果不错

## 测试 XGBoost

In [9]:
trainset_1 = xgb.DMatrix(train_1, label=labels_1)
testset_1 = xgb.DMatrix(test_1)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L2 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_1 = xgb.train(params, trainset_1, num_boost_round=4000)
predict_xgb_1 = model_1.predict(testset_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - predict_xgb_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 120.68144567678694


In [11]:
trainset_2 = xgb.DMatrix(train_2, label=labels_2)
testset_2 = xgb.DMatrix(test_2)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L2 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_2 = xgb.train(params, trainset_2, num_boost_round=4000)
predict_xgb_2 = model_2.predict(testset_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - predict_xgb_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 122.20946253396676


In [12]:
trainset_3 = xgb.DMatrix(train_3, label=labels_3)
testset_3 = xgb.DMatrix(test_3)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L3 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_3 = xgb.train(params, trainset_3, num_boost_round=4000)
predict_xgb_3 = model_3.predict(testset_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - predict_xgb_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 117.56370604813299


In [31]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 156.572741106


* 选取全特征, 加上同比/环比 数据, XGBoost 模型效果较差, 不使用这样的特征

# 选用带时间窗口, 不带同比/环比数据集 特征集测试

In [34]:
data = pd.read_csv('../../raw/LiChuan/trainallfeatures.csv')
submit = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

In [35]:
# 去掉 2012 年数据
data = data[data['year'] != 2012]
labels = data['sale_quantity']
data = data.drop(['sale_date', 'class_id', 'sale_quantity'], axis=1)

In [36]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = data[:-560]
test_1 = data[-560:-420]
labels_1 = labels[:-560]
test_labels_1 = labels[-560:-420]

# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = data[:-420]
test_2 = data[-420:-280]
labels_2 = labels[:-420]
test_labels_2 = labels[-420:-280]

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = data[:-280]
test_3 = data[-280:-140]
labels_3 = labels[:-280]
test_labels_3 = labels[-280:-140]

## 测试 LightGBM

In [38]:
lgb_train_1 = lgb.Dataset(train_1, labels_1)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_1 = lgb.train(params, lgb_train_1, 900)
lgb_pred_1 = model_1.predict(test_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - lgb_pred_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 145.116054859


In [39]:
lgb_train_2 = lgb.Dataset(train_2, labels_2)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_2 = lgb.train(params, lgb_train_2, 900)
lgb_pred_2 = model_2.predict(test_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - lgb_pred_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 116.378154374


In [40]:
lgb_train_3 = lgb.Dataset(train_3, labels_3)

params = {
'learning_rate': 0.003,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 30,
'min_hessian': 1,
'verbose': -1,}

model_3 = lgb.train(params, lgb_train_3, 900)
lgb_pred_3 = model_3.predict(test_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - lgb_pred_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 150.31146246


In [41]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 137.268557231


* LightGBM 效果不如加上同比/环比 特征

## 测试 XGBoost

In [42]:
trainset_1 = xgb.DMatrix(train_1, label=labels_1)
testset_1 = xgb.DMatrix(test_1)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L2 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_1 = xgb.train(params, trainset_1, num_boost_round=4000)
predict_xgb_1 = model_1.predict(testset_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - predict_xgb_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 193.746542278


In [43]:
trainset_2 = xgb.DMatrix(train_2, label=labels_2)
testset_2 = xgb.DMatrix(test_2)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L2 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_2 = xgb.train(params, trainset_2, num_boost_round=4000)
predict_xgb_2 = model_2.predict(testset_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - predict_xgb_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 135.365487254


In [44]:
trainset_3 = xgb.DMatrix(train_3, label=labels_3)
testset_3 = xgb.DMatrix(test_3)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L3 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_3 = xgb.train(params, trainset_3, num_boost_round=4000)
predict_xgb_3 = model_3.predict(testset_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - predict_xgb_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 131.289320436


In [45]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 153.467116656


* 比上面加同比/环比, 略微强一点

# 不带时间窗口特征

In [46]:
data = pd.read_csv('../../raw/LiChuan/trainSaleDate.csv')
submit = pd.read_csv('../../raw/CarsSaleForecast/yancheng_testA_20171225.csv')

In [52]:
# 去掉 2012 年数据
data = data[data['year'] != 2012]
labels = data['sale_quantity']
data = data.drop(['sale_date', 'class_id', 'sale_quantity'], axis=1)

In [54]:
# 训练集 2013-01 至 2017-07, 测试集 2017-08
train_1 = data[:-420]
test_1 = data[-420:-280]
labels_1 = labels[:-420]
test_labels_1 = labels[-420:-280]

# 训练集 2013-01 至 2017-08, 测试集 2017-09
train_2 = data[:-280]
test_2 = data[-280:-140]
labels_2 = labels[:-280]
test_labels_2 = labels[-280:-140]

# 训练集 2013-01 至 2017-09, 测试集 2017-10
train_3 = data[:-140]
test_3 = data[-140:]
labels_3 = labels[:-140]
test_labels_3 = labels[-140:]

## 测试 LightGBM

In [56]:
lgb_train_1 = lgb.Dataset(train_1, labels_1)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_1 = lgb.train(params, lgb_train_1, 900)
lgb_pred_1 = model_1.predict(test_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - lgb_pred_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 152.625651074


In [57]:
lgb_train_2 = lgb.Dataset(train_2, labels_2)

params = {
'learning_rate': 0.002,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 20,
'min_hessian': 1,
'verbose': -1,}

model_2 = lgb.train(params, lgb_train_2, 900)
lgb_pred_2 = model_2.predict(test_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - lgb_pred_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 120.574974352


In [58]:
lgb_train_3 = lgb.Dataset(train_3, labels_3)

params = {
'learning_rate': 0.003,
'boosting_type': 'gbdt',
'objective': 'regression',
'metric': 'rmse',
'sub_feature': 0.8,
'num_leaves': 35,
'min_data': 30,
'min_hessian': 1,
'verbose': -1,}

model_3 = lgb.train(params, lgb_train_3, 900)
lgb_pred_3 = model_3.predict(test_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - lgb_pred_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 160.703307873


In [59]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 144.634644433


## 测试 XGBoost

In [60]:
trainset_1 = xgb.DMatrix(train_1, label=labels_1)
testset_1 = xgb.DMatrix(test_1)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L2 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_1 = xgb.train(params, trainset_1, num_boost_round=4000)
predict_xgb_1 = model_1.predict(testset_1)
rmsetmp_1 = sp.sqrt(sp.mean((test_labels_1 - predict_xgb_1) ** 2))

print('This time rmse is: '+ str(rmsetmp_1))

This time rmse is: 140.010796927


In [61]:
trainset_2 = xgb.DMatrix(train_2, label=labels_2)
testset_2 = xgb.DMatrix(test_2)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L2 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_2 = xgb.train(params, trainset_2, num_boost_round=4000)
predict_xgb_2 = model_2.predict(testset_2)
rmsetmp_2 = sp.sqrt(sp.mean((test_labels_2 - predict_xgb_2) ** 2))

print('This time rmse is: '+ str(rmsetmp_2))

This time rmse is: 163.353862481


In [62]:
trainset_3 = xgb.DMatrix(train_3, label=labels_3)
testset_3 = xgb.DMatrix(test_3)

params = {
    'booster': 'gbtree', # 基于树的模型进行计算
    'objective': 'reg:linear', # 线性回归
    'eval_metric': 'rmse', # RMSE 评价函数
    'gamma': 0.1, # 在树的叶子节点上进一步划分所需的最小损失减少。算法越大，越保守。
    'min_child_weight': 1.1, # 孩子节点中最小的样本权重和。如果一个叶子节点的样本权重和小于min_child_weight则拆分过程结束。
    'max_depth': 5, # 树的最大深度。缺省值为6
    'lambda': 10, # L3 正则的惩罚系数
    'subsample': 0.8, # 用于训练模型的子样本占整个样本集合的比例。
    'colsample_bytree': 0.8, # 在建立树时对特征采样的比例。
    'tree_method': 'exact' # 树的构造算法-Exact greedy algorithm(确切贪心算法)
    }
    
model_3 = xgb.train(params, trainset_3, num_boost_round=4000)
predict_xgb_3 = model_3.predict(testset_3)
rmsetmp_3 = sp.sqrt(sp.mean((test_labels_3 - predict_xgb_3) ** 2))

print('This time rmse is: '+ str(rmsetmp_3))

This time rmse is: 152.222508663


In [63]:
print('The average rmse is: '+ str((rmsetmp_1 + rmsetmp_2 + rmsetmp_3)/3))

The average rmse is: 151.862389357


* XGBoost 效果似乎也不咋地

In [69]:
rmse_1 = sp.sqrt(sp.mean((lgb_pred_1 - predict_xgb_1) ** 2))

rmse_2 = sp.sqrt(sp.mean((lgb_pred_2 - predict_xgb_2) ** 2))

rmse_3 = sp.sqrt(sp.mean((lgb_pred_3 - predict_xgb_3) ** 2))

print(rmse_1, rmse_2, rmse_3)

115.838369395 125.853569134 89.5277146076
